In [1]:
from biocrnpyler import *
import numpy as np
import pylab as plt
from subsbml import *
import libsbml

# fusion sub1

In [32]:
from biocrnpyler import *
import numpy as np
import pylab as plt

exsA = Species ("exsA") 
repressor = Species("tetR")
dna_f= Species ("dna_f")
rna_f= Species ("rna_f")
ATP= Species ("ATP")
ADP= Species ("ADP")
atc_i = Species("atc_i_cell2") # material_type = "ligand")
p= Species("P")
d_f= ComplexSpecies([dna_f,repressor])
d_f_atc= ComplexSpecies([dna_f,repressor,atc_i])

#Create a ChemicalComplex to model ligand-repressor bindning
inactive_repressor = ChemicalComplex([repressor, atc_i])

species=[dna_f, repressor, rna_f, exsA, ATP,ADP, atc_i, inactive_repressor.get_species(), d_f, d_f_atc, p]

rex1=Reaction([atc_i, repressor], [inactive_repressor.get_species()], propensity_type = "massaction",k= 2e-5, k_rev=2e-9)
rex2=Reaction([dna_f, repressor], [d_f], propensity_type = "massaction", k= 4e-7, k_rev= 1e-12) #1e+11 #1
rex3=Reaction([d_f,atc_i ], [d_f_atc], propensity_type = "massaction",k= 6e-8, k_rev= 3e-9 ) # 6.5e+7 #10
rex4=Reaction([d_f_atc], [dna_f, inactive_repressor.get_species()], propensity_type = "massaction", k= 5e-5, k_rev=1e-10 ) #1000,10
rex5=Reaction([dna_f], [dna_f, rna_f], propensity_type = "massaction", k=.001875)  #{"k":18.75, "n":2, "K":20, "s1":repressor, "d":dna_g}
rex6=Reaction([rna_f, ATP], [rna_f, ADP, exsA,p], propensity_type = "proportionalhillpositive", propensity_params = {"k":0.0005, "n":1, "K":1, "s1":ATP, "d":rna_f}) 
rex7=Reaction([rna_f], [], propensity_type = "massaction", k= .001) 
#rex8=Reaction([exsA], [], propensity_type = "massaction", k= 1e-5)

r=[rex1, rex2, rex3, rex4, rex5, rex6, rex7 ]

CRN1 = ChemicalReactionNetwork(species = species, reactions = r)

sub1 = CRN1.write_sbml_file('Models/exsA_expression.xml')
print('Model written to a SBML file successfully')



Model written to a SBML file successfully


In [33]:
CRN1.generate_sbml_model()

(<SBMLDocument>, <Model biocrnpyler_986640 "biocrnpyler_986640">)

## Changing reaction name in order for it to be present while combining the models

In [34]:
v1,v2=CRN1.generate_sbml_model()
v2.getElementBySId('r0').setName("r0cell2")
v2.getElementBySId('r0_1').setName("r0_1cell2")
libsbml.writeSBML(v1, 'Models/fusion_sub1.xml')

1

In [35]:
# Import statements
from subsbml import createNewSubsystem, createSubsystem

import numpy as np
import pylab as plt
# Import SBML models by creating Subsystem class objects
ss1 = createSubsystem('Models/fusion_sub1.xml', subsystemName = 'exsA')
ss2 = createSubsystem('Models/ATP_rheostat.xml', subsystemName = 'ATP_rheostat')

ss2.renameSName('metabolite_atp', 'ATP')
ss2.renameSName('metabolite_adp', 'ADP')
ss2.renameSName('metabolite_pi', 'P')


combsub = createNewSubsystem()
combsub.combineSubsystems([ss1,ss2])

# Set signal concentration (input) - manually and get ID for protein X
_ = combsub.writeSBML('Models/comb_exsA_sub.xml')


# fusion sub2

In [36]:
from biocrnpyler import *
import numpy as np
import pylab as plt

activator = Species ("exsA") 
dna_h= Species ("dna_h")
rna_h= Species ("rna_h")
ATP= Species ("ATP")
ADP= Species ("ADP")
p= Species("P")
d_a= ComplexSpecies([dna_h,activator])
gfp= Species("gfp")

species2=[dna_h, rna_h, activator, ATP, ADP, p, d_a, gfp]

r1=Reaction([dna_h, activator], [d_a], propensity_type = "massaction", k= 4e-5, k_rev= 1e-12) 
r2=Reaction([d_a], [d_a, rna_h], propensity_type =  "massaction", k=.001875)  #{"k":18.75, "n":2, "K":20, "s1":repressor, "d":dna_g}
#r3=Reaction([dna_h], [dna_h, rna_h], propensity_type = "massaction", k= .000001) #leak
r4=Reaction([rna_h, ATP], [rna_h, ADP, gfp, p], propensity_type = "proportionalhillpositive", propensity_params = {"k":0.0005, "n":1, "K":1, "s1":ATP, "d":rna_h}) 
r5=Reaction([rna_h], [], propensity_type = "massaction", k= .001) 
#r6=Reaction([gfp], [], propensity_type = "massaction", k= 1e-5)


rex=[r1, r2, r4, r5]

CRN2 = ChemicalReactionNetwork(species = species2, reactions = rex)

sub2 = CRN2.write_sbml_file('Models/fusion_sub2.xml')
print('Model written to a SBML file successfully')



Model written to a SBML file successfully


In [37]:
# Import statements
from subsbml import createNewSubsystem, createSubsystem

import numpy as np
import pylab as plt
# Import SBML models by creating Subsystem class objects
ss1 = createSubsystem('Models/fusion_sub2.xml', subsystemName = 'gfp')
ss2 = createSubsystem('Models/ATP_rheostat.xml', subsystemName = 'ATP_rheostat')

ss2.renameSName('metabolite_atp', 'ATP')
ss2.renameSName('metabolite_adp', 'ADP')
ss2.renameSName('metabolite_pi', 'P')


combsub = createNewSubsystem()
combsub.combineSubsystems([ss1,ss2])

# Set signal concentration (input) - manually and get ID for protein X
_ = combsub.writeSBML('Models/comb_gfp1_sub.xml')


## specific atc  membrane model

In [38]:
##### Creating a basic membrane model for IPTG transport####

from subsbml import createNewSubsystem, SimpleModel, SimpleReaction

atc_mb_ss2 = createNewSubsystem()
model2 = atc_mb_ss2.createNewModel('atc_membrane2','second','mole','substance')

atc_mb2 = SimpleModel(model2)

per_second = atc_mb2.createNewUnitDefinition('per_second',libsbml.UNIT_KIND_SECOND,-1,0,1)
substance = atc_mb2.createNewUnitDefinition('substance',libsbml.UNIT_KIND_DIMENSIONLESS, 1, 0, 1)

################# Two important lines of code when creating membrane #########################
atc_mb2.createNewCompartment('internal','internal',1e-6,'litre',True)
atc_mb2.createNewCompartment('external','external',1e-6,'litre',True)
##############################################################################################

atc_mb2.createNewSpecies( 'atc_e','external',1e4,False)
atc_mb2.createNewSpecies( 'atc_i_cell2','internal',0,False)
atc_mb2.createNewSpecies( 'ssDNA_comp1','internal',0,False)

atc_mb2.createNewParameter( 'kb',2e-3,False,'per_second')
atc_mb2.createNewParameter( 'kd',1e-5,False,'per_second')

r1_sbml = model2.createReaction()
r1 = SimpleReaction(r1_sbml)
r1.setId('r1_cell2')
r1.setReversible(True)
r1.createNewReactant('atc_e',False,1)
r1.createNewProduct('atc_i_cell2', False, 1)
r1.createRate('kb * atc_e - kd * atc_i_cell2')

# Write to SBML file 
_ = atc_mb_ss2.writeSBML('Models/atc_membrane_cell2.xml')
print('Model written to a SBML file successfully')

Model written to a SBML file successfully


In [39]:
##### Creating a basic membrane model for IPTG transport####

from subsbml import createNewSubsystem, SimpleModel, SimpleReaction

atc_mb_ss3 = createNewSubsystem()
model3 = atc_mb_ss3.createNewModel('atc_membrane3','second','mole','substance')

atc_mb3 = SimpleModel(model3)

per_second = atc_mb3.createNewUnitDefinition('per_second',libsbml.UNIT_KIND_SECOND,-1,0,1)
substance = atc_mb3.createNewUnitDefinition('substance',libsbml.UNIT_KIND_DIMENSIONLESS, 1, 0, 1)

################# Two important lines of code when creating membrane #########################
atc_mb3.createNewCompartment('internal','internal',1e-6,'litre',True)
atc_mb3.createNewCompartment('external','external',1e-6,'litre',True)
##############################################################################################

atc_mb3.createNewSpecies( 'atc_e','external',1e4,False)
atc_mb3.createNewSpecies( 'atc_i_cell3','internal',0,False)
atc_mb3.createNewSpecies( 'ssDNA_comp2','internal',0,False)


atc_mb3.createNewParameter( 'kb',2e-3,False,'per_second')
atc_mb3.createNewParameter( 'kd',1e-5,False,'per_second')

r1_sbml = model3.createReaction()
r1 = SimpleReaction(r1_sbml)
r1.setId('r1_cell3')
r1.setReversible(True)
r1.createNewReactant('atc_e',False,1)
r1.createNewProduct('atc_i_cell3', False, 1)
r1.createRate('kb * atc_e - kd * atc_i_cell3')

# Write to SBML file 
_ = atc_mb_ss3.writeSBML('Models/atc_membrane_cell3.xml')
print('Model written to a SBML file successfully')

Model written to a SBML file successfully
